In [2]:
import torch
import torch.nn as nn
import torch.optim as optim



In [6]:
##Bradley TErry Model

import torch
import torch.nn as nn
import torch.optim as optim
class BTModel(nn.Module):
    def __init__(self, N):
        super(BTModel, self).__init__()
        self.reward = nn.Parameter(torch.ones(N))
        
    def forward_exp(self, chosen_id, rejected_id):
        reward_chosen = torch.exp(self.reward[chosen_id])
        reward_rejected = torch.exp(self.reward[rejected_id])
        return reward_chosen / (reward_chosen + reward_rejected)

    def forward_sigmoid(self, chosen_id, rejected_id):
        reward_chosen = self.reward[chosen_id]
        reward_rejected = self.reward[rejected_id]
        return torch.sigmoid(reward_chosen - reward_rejected)

    def loss(self, pred, label):
        return -torch.log(pred) if label == 1 else -torch.log(1 - pred)

In [7]:
model = BTModel(N)
datas = [(0, 1, 1), (2, 3, 1), (1, 3, 1)]
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
loss_fn = nn.BCELoss()
for i in range(100):
    total_loss = 0
    for data in datas:
        id_i, id_j, label = data
        optimizer.zero_grad()
        pred = model.forward_sigmoid(id_i, id_j)
        loss = model.loss(pred, torch.tensor(label, dtype=torch.float32))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if i%10==0 : print(f"Epoch {i}, Loss: {total_loss}")

print(model.reward)

Epoch 0, Loss: 2.079441547393799
Epoch 10, Loss: 1.937548816204071
Epoch 20, Loss: 1.811079204082489
Epoch 30, Loss: 1.6980656385421753
Epoch 40, Loss: 1.5967631340026855
Epoch 50, Loss: 1.5056480765342712
Epoch 60, Loss: 1.4234036803245544
Epoch 70, Loss: 1.3488986492156982
Epoch 80, Loss: 1.281165212392807
Epoch 90, Loss: 1.2193749248981476
Parameter containing:
tensor([1.4402, 0.9630, 1.3558, 0.2410, 1.0000], requires_grad=True)
